# Global settings

In [1]:
device = 'cuda'
bs = 64
test_pct = .10
val_pct = .20
epochs = 2

# Data proc

In [2]:
import pandas as pd
import numpy as np
import torch
import itertools
from make_matrix import *
from torch.utils.data import TensorDataset, DataLoader, Sampler
import torch.nn as nn
import torch.nn.functional as F

In [3]:
gidf = pd.read_csv('gi_dset.csv')

In [4]:
gidf.head()

,gene,effector,zscore
0,AME1,BME0304,-2.416447
1,AME1,BME0390,-3.338498
2,AME1,BME0736,-2.054660
3,AME1,BME1013,-4.013577
4,AME1,BME1044,-2.855403


### Numericalize proteins

In [5]:
def numericalize(df):
    genes = df['gene'].unique()
    effectors = df['effector'].unique()
    gs_map = {v:i for i,v in enumerate(genes)}
    es_map = {h:i for i,h in enumerate(effectors)}
    
    df.gene = df.gene.apply(lambda x : gs_map[x])
    df.effector = df.effector.apply(lambda x : es_map[x])
    df = df.sample(frac=1).reset_index()
    return df[['gene', 'effector', 'zscore']], gs_map, es_map

In [6]:
gidf, gs_map, es_map = numericalize(gidf)

In [7]:
gidf.head()

,gene,effector,zscore
0,2210,24,-0.602251
1,2012,20,0.508293
2,2629,10,0.226026
3,221,6,1.254141
4,3904,15,-0.539812


### Make a dataloaders

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
def arr_to_torch(arr, dtype):
    t =  torch.from_numpy(np.array(arr)).type(dtype).to(device)
    return t

In [10]:
def getLoaders(df):
    X = list(zip(df.gene.values, df.effector.values))
    y = df['zscore'].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_pct, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_pct, random_state=1)
    
    train_dset = TensorDataset(arr_to_torch(X_train, torch.long), arr_to_torch(y_train, torch.float32))
    train_loader = DataLoader(train_dset, batch_size=bs, shuffle=True)
    
    val_dset = TensorDataset(arr_to_torch(X_val, torch.long), arr_to_torch(y_val, torch.float32))
    val_loader = DataLoader(val_dset, batch_size=bs, shuffle=True)
    
    test_dset = TensorDataset(arr_to_torch(X_test, torch.long), arr_to_torch(y_test, torch.float32))
    test_loader = DataLoader(test_dset, batch_size=bs, shuffle=True)
    
    return train_loader, val_loader, test_loader

In [11]:
train_loader, val_loader, test_loader = getLoaders(gidf)

# Model

In [12]:
def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss

In [13]:
class MF(nn.Module):
    def __init__(self, n_virus, n_human, k=18, c_vector=1.0, c_bias=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_virus = n_virus
        self.n_human = n_human
        self.c_bias = c_bias
        self.c_vector = c_vector
        
        self.virus = nn.Embedding(n_virus, k)
        self.human = nn.Embedding(n_human, k)
        
        # We've added new terms here:
        self.bias_virus = nn.Embedding(n_virus, 1)
        self.bias_human = nn.Embedding(n_human, 1)
        self.bias = nn.Parameter(torch.ones(1))
    
    def forward(self, train_x):
        virus_id = train_x[:, 0]
        human_id = train_x[:, 1]
        vector_virus = self.virus(virus_id)
        vector_human = self.human(human_id)
        
        # Pull out biases
        bias_virus = self.bias_virus(virus_id).squeeze()
        bias_human = self.bias_human(human_id).squeeze()
        biases = (self.bias + bias_virus + bias_human)
        
        ui_interaction = torch.sum(vector_virus * vector_human, dim=1)
        
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
#         loss_mse = F.binary_cross_entropy_with_logits(prediction, target.squeeze())
        loss_mse = F.mse_loss(prediction, target.squeeze())
    
        # Add new regularization to the biases
        prior_bias_virus =  l2_regularize(self.bias_virus.weight) * self.c_bias
        prior_bias_human = l2_regularize(self.bias_human.weight) * self.c_bias
        
        prior_virus =  l2_regularize(self.virus.weight.data) * self.c_vector
        prior_human = l2_regularize(self.human.weight.data) * self.c_vector
        total = loss_mse + prior_virus + prior_human + prior_bias_virus + prior_bias_human
        return total

# Trainer

In [14]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Accuracy, Precision, Recall
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError, Loss
from ignite.contrib.metrics import AveragePrecision, ROC_AUC

from datetime import datetime

In [15]:
class Trainer:
    
    def __init__(self, model, crit, optim, writer, train_loader, val_loader, test_loader, modelname):
        self.model = model
        self.crit = crit
        self.optim = optim
        self.writer = writer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.best_model = None
        self.best_acc = 999
        self.modelname = modelname
        
        self.trainer = create_supervised_trainer(model, optim, crit)
        self.metrics = {'loss': Loss(crit), "acc": MeanSquaredError()}
        self.evaluator = create_supervised_evaluator(model, metrics=self.metrics)
        
        ## Add events
        self.trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=self.log_training_loss)
        self.trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=self.log_validation_results)
        self.trainer.add_event_handler(event_name=Events.COMPLETED, handler=self.log_test_results)
        
        print(model)
        
    def log_training_loss(self, engine, log_interval=400):
        epoch = engine.state.epoch
        itr = engine.state.iteration
        fmt = "TRAIN: Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
        msg = fmt.format(epoch, itr, len(self.train_loader), engine.state.output)
        self.model.itr = itr
        if itr % log_interval == 0:
            print(msg)
#             self.evaluator.run(self.train_loader)
            
#             metrics = self.evaluator.state.metrics
#             mse = metrics['loss']
#             ap = metrics['ap']
#             roc = metrics['roc']
            
#             print("Epoch[{}] Validation MSE: {:.2f} Avg Prec: {:.2f} ROC: {:.2f} "
#                   .format(engine.state.epoch, mse, ap, roc))
            
#             self.writer.add_scalar("training/mse", mse, engine.state.epoch)
#             self.writer.add_scalar("training/ap", ap, engine.state.epoch)
#             self.writer.add_scalar("training/roc", roc, engine.state.epoch)
            
    def log_validation_results(self, engine):
        self.evaluator.run(self.val_loader)
        
        metrics = self.evaluator.state.metrics
        mse = metrics['loss']
        acc = metrics['acc']
        
        if acc < self.best_acc:
            self.best_acc = acc
            self.best_model = model.state_dict()
        
        print("VALIDATION Epoch[{}] Validation MSE: {:.2f} Acc: {:.2f}"
              .format(engine.state.epoch, mse, acc))
        self.writer.add_scalar("validation/mse", mse, engine.state.epoch)
        self.writer.add_scalar("validation/acc", acc, engine.state.epoch)
        
    def log_test_results(self, engine):
        self.evaluator.run(self.test_loader)
        
        metrics = self.evaluator.state.metrics
        mse = metrics['loss']
        acc = metrics['acc']


        print("TEST: Epoch[{}] Validation MSE: {:.2f} Acc: {:.2f}"
              .format(engine.state.epoch, mse, acc))
        
        print("BEST Acc: ", self.best_acc)
        torch.save(self.best_model, './{}.pt'.format(self.modelname))
        
    def run(self, epochs):
        self.trainer.run(self.train_loader, max_epochs=epochs)

## Hyperparametrs

In [16]:
lr = 1e-3
k =20
# regularizing bias
c_bias = 1e-4
c_vector = 1e-4
batchsize = bs

In [17]:
log_dir = 'runs/FLU_simple_mf_02_bias_' + str(datetime.now()).replace(' ', '_')
print(log_dir)

runs/FLU_simple_mf_02_bias_2019-05-19_23:34:59.604969


In [18]:
writer = SummaryWriter(log_dir=log_dir)
model = MF(len(gs_map), len(es_map), writer=writer, k=k, c_bias=c_bias, c_vector=c_vector)
crit = model.loss
model.cuda()
optim = torch.optim.Adam(model.parameters())

In [19]:
trainer = Trainer(model, crit, optim, writer, train_loader, val_loader, test_loader, 'flumodel')

MF(
  (virus): Embedding(4450, 20)
  (human): Embedding(36, 20)
  (bias_virus): Embedding(4450, 1)
  (bias_human): Embedding(36, 1)
)


In [20]:
trainer.run(2)

TRAIN: Epoch[1] Iteration[400/1897] Loss: 28.52
TRAIN: Epoch[1] Iteration[800/1897] Loss: 23.36
TRAIN: Epoch[1] Iteration[1200/1897] Loss: 20.53
TRAIN: Epoch[1] Iteration[1600/1897] Loss: 21.19
VALIDATION Epoch[1] Validation MSE: 19.49 Acc: 10.70
TRAIN: Epoch[2] Iteration[2000/1897] Loss: 17.09
TRAIN: Epoch[2] Iteration[2400/1897] Loss: 13.82
TRAIN: Epoch[2] Iteration[2800/1897] Loss: 14.97
TRAIN: Epoch[2] Iteration[3200/1897] Loss: 15.68
TRAIN: Epoch[2] Iteration[3600/1897] Loss: 14.66
VALIDATION Epoch[2] Validation MSE: 13.91 Acc: 5.49
TEST: Epoch[2] Validation MSE: 13.87 Acc: 5.46
BEST Acc:  5.493472703468645
